In [21]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, models
import cv2
from PIL import Image
import os
import requests # Import the requests library

# For better performance, enable mixed precision if available
try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled")
except:
    print("Using default precision")

class NeuralStyleTransfer:
    def __init__(self, content_path, style_path, img_size=512):
        """
        Initialize Neural Style Transfer

        Args:
            content_path: Path to content image
            style_path: Path to style image (artwork)
            img_size: Size to resize images to
        """
        self.img_size = img_size
        self.content_image = self.load_and_preprocess_image(content_path)
        self.style_image = self.load_and_preprocess_image(style_path)

        # Use VGG19 for feature extraction
        self.model = self.get_vgg19_model()

        # Style layers and their weights
        self.style_layer_names = [
            'block1_conv1',  # Low-level features (edges, textures)
            'block2_conv1',  # Mid-level patterns
            'block3_conv1',  # Higher-level patterns
            'block4_conv1',  # Object parts
            'block5_conv1'   # Object-level features
        ]

        # Weights for style layers
        self.style_layer_weights = [0.5, 1.0, 1.5, 3.0, 4.0]

        # Content layer (deeper layer captures content)
        self.content_layer_name = 'block4_conv2'

        # Weights for loss components
        self.content_weight = 1e-3
        self.style_weight = 1e-1
        self.tv_weight = 1e-4  # Total variation weight for smoothness

    def load_and_preprocess_image(self, path):
        """Load and preprocess image for VGG19"""
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.convert_image_dtype(img, tf.float32)

        # Resize while maintaining aspect ratio
        shape = tf.shape(img)[:-1]
        scale = tf.cast(self.img_size, tf.float32) / tf.cast(tf.maximum(shape[0], shape[1]), tf.float32)
        new_shape = tf.cast(tf.cast(shape, tf.float32) * scale, tf.int32)
        img = tf.image.resize(img, new_shape)

        # Center crop to square
        img = tf.image.resize_with_crop_or_pad(img, self.img_size, self.img_size)

        # Add batch dimension
        img = tf.expand_dims(img, 0)

        # Convert to VGG19 format (mean subtraction)
        img = tf.keras.applications.vgg19.preprocess_input(img * 255)

        return img

    def deprocess_image(self, img):
        """Convert preprocessed image back to displayable format"""
        img = img.numpy().squeeze()

        # Reverse VGG19 preprocessing
        img[:, :, 0] += 103.939
        img[:, :, 1] += 116.779
        img[:, :, 2] += 123.68

        # BGR to RGB and clip
        img = img[:, :, ::-1]
        img = np.clip(img, 0, 255).astype('uint8')

        return img

    def get_vgg19_model(self):
        """Create VGG19 model with intermediate layer outputs"""
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        # Get outputs for style and content layers
        style_outputs = [vgg.get_layer(name).output for name in self.style_layer_names]
        content_outputs = [vgg.get_layer(self.content_layer_name).output]

        # Combine all outputs
        outputs = style_outputs + content_outputs

        model = tf.keras.Model(vgg.input, outputs)

        return model

    def gram_matrix(self, feature_map):
        """Compute Gram matrix for style representation"""
        # Reshape feature map: (height, width, channels) -> (height * width, channels)
        shape = tf.shape(feature_map)
        channels = shape[-1]
        a = tf.reshape(feature_map, [-1, channels])

        # Compute Gram matrix
        gram = tf.matmul(a, a, transpose_a=True)

        # Normalize by number of locations
        gram = gram / tf.cast(tf.reduce_prod(shape[:-1]), tf.float32)

        return gram

    def compute_loss(self, generated_image, content_features, style_features):
        """
        Compute total loss: content loss + style loss + total variation loss
        """
        # Get features for generated image
        generated_features = self.model(generated_image)

        # Split into style and content features
        gen_style_features = generated_features[:len(self.style_layer_names)]
        gen_content_features = generated_features[len(self.style_layer_names):]

        # Content loss (L2 distance between content features)
        content_loss = 0
        for gen_feat, content_feat in zip(gen_content_features, content_features):
            content_loss += tf.reduce_mean(tf.square(gen_feat - content_feat))

        # Style loss (L2 distance between Gram matrices)
        style_loss = 0
        for gen_feat, style_feat, weight in zip(gen_style_features, style_features, self.style_layer_weights):
            # Compute Gram matrices
            gen_gram = self.gram_matrix(gen_feat)
            style_gram = self.gram_matrix(style_feat)

            # Compute loss for this layer
            layer_loss = tf.reduce_mean(tf.square(gen_gram - style_gram))
            style_loss += weight * layer_loss

        # Total variation loss for smoothness
        tv_loss = tf.image.total_variation(generated_image)

        # Combine losses
        total_loss = (self.content_weight * content_loss +
                     self.style_weight * style_loss +
                     self.tv_weight * tv_loss)

        return total_loss, content_loss, style_loss, tv_loss

    def transfer_style(self, num_iterations=1000, learning_rate=0.02, save_progress=True):
        """
        Perform neural style transfer

        Args:
            num_iterations: Number of optimization iterations
            learning_rate: Learning rate for optimization
            save_progress: Whether to save intermediate results

        Returns:
            Final stylized image
        """
        print("Starting neural style transfer...")

        # Precompute content and style features
        content_features = self.model(self.content_image)
        content_features = content_features[len(self.style_layer_names):]

        style_features = self.model(self.style_image)
        style_features = style_features[:len(self.style_layer_names)]

        # Initialize generated image with content image
        generated_image = tf.Variable(self.content_image, dtype=tf.float32)

        # Optimizer
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

        # Training loop
        for i in range(num_iterations):
            with tf.GradientTape() as tape:
                tape.watch(generated_image)
                total_loss, c_loss, s_loss, tv = self.compute_loss(
                    generated_image, content_features, style_features
                )

            # Compute gradients and update
            gradients = tape.gradient(total_loss, generated_image)
            optimizer.apply_gradients([(gradients, generated_image)])

            # Clip values to valid range
            generated_image.assign(tf.clip_by_value(generated_image, 0, 255))

            # Print progress
            if i % 100 == 0:
                print(f"Iteration {i}: Total Loss = {total_loss:.4f}, "
                      f"Content = {c_loss:.4f}, Style = {s_loss:.4f}, TV = {tv:.4f}")

                # Save intermediate result
                if save_progress:
                    result = self.deprocess_image(generated_image)
                    plt.figure(figsize=(8, 8))
                    plt.imshow(result)
                    plt.title(f'Iteration {i}')
                    plt.axis('off')
                    plt.savefig(f'progress_iter_{i}.png', bbox_inches='tight', pad_inches=0)
                    plt.close()

        # Final result
        final_image = self.deprocess_image(generated_image)

        return final_image

    def display_results(self, final_image):
        """Display content, style, and final images"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # Content image
        content_display = self.deprocess_image(self.content_image)
        axes[0].imshow(content_display)
        axes[0].set_title('Content Image')
        axes[0].axis('off')

        # Style image
        style_display = self.deprocess_image(self.style_image)
        axes[1].imshow(style_display)
        axes[1].set_title('Style Image (Artwork)')
        axes[1].axis('off')

        # Final image
        axes[2].imshow(final_image)
        axes[2].set_title('Stylized Image')
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()

        # Save final image
        Image.fromarray(final_image).save('stylized_result.png')
        print("Final image saved as 'stylized_result.png'")

# Example usage
def download_sample_images():
    """Download sample images if they don't exist"""
    # import urllib.request # No longer needed, using requests

    # Sample images (you can replace these with your own)
    images = {
        'content.jpg': 'https://raw.githubusercontent.com/pytorch/examples/main/fast_neural_style/images/content-images/neural_style_dancing.jpg',
        'style.jpg': 'https://raw.githubusercontent.com/pytorch/examples/main/fast_neural_style/images/style-images/neural_style_picasso.jpg'
    }

    # Define a User-Agent header to mimic a web browser
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    for filename, url in images.items():
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f"Successfully downloaded {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading {filename}: {e}")
                print("Please check the URL or provide local image files.")

# Main execution
if __name__ == "__main__":
    # Download sample images
    download_sample_images()

    # Create style transfer object
    stylizer = NeuralStyleTransfer(
        content_path='content.jpg',
        style_path='style.jpg',
        img_size=512
    )

    # Perform style transfer
    final_image = stylizer.transfer_style(
        num_iterations=1000,
        learning_rate=0.02,
        save_progress=True
    )

    # Display results
    stylizer.display_results(final_image)

Mixed precision enabled
Error downloading content.jpg: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/pytorch/examples/main/fast_neural_style/images/content-images/neural_style_dancing.jpg
Please check the URL or provide local image files.
Error downloading style.jpg: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/pytorch/examples/main/fast_neural_style/images/style-images/neural_style_picasso.jpg
Please check the URL or provide local image files.


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} content.jpg; No such file or directory [Op:ReadFile]